In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
import torch
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
!git clone https://github.com/fabio-bays/the-dip.git

Cloning into 'the-dip'...
remote: Enumerating objects: 1794, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1794 (delta 21), reused 34 (delta 17), pack-reused 1754 (from 1)
Receiving objects: 100% (1794/1794), 2.11 GiB | 29.97 MiB/s, done.
Resolving deltas: 100% (251/251), done.
Updating files: 100% (1614/1614), done.


In [10]:

# Caminho para o diretório principal com as imagens
base_dir = "/content/the-dip/augmented_dataset"

# Carregar o modelo YOLOv8
model = YOLO("yolov8n.pt")  # Pode usar yolov8s.pt, yolov8m.pt, etc.

# Lista para armazenar os feature vectors e os nomes das imagens
feature_vectors = []
image_names = []

# Percorrer todas as imagens dentro dos subdiretórios
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):  # Verifica se é uma imagem
            img_path = os.path.join(root, file)
            image_names.append(img_path)

            # Carregar e pré-processar a imagem
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # YOLO espera RGB
            img_resized = cv2.resize(img_rgb, (640, 640))  # Redimensiona para o tamanho esperado pelo modelo

            # Converter para tensor
            img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
            img_tensor = img_tensor.unsqueeze(0)  # Adiciona dimensão batch

            # Extrair feature vector usando o backbone do YOLOv8
            backbone = model.model.model[0]  # Acesso ao backbone do modelo
            with torch.no_grad():
                features = backbone(img_tensor)

            # Aplicar Global Average Pooling para obter um vetor 1D
            feature_vector = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1)).reshape(features.size(0), -1)
            feature_vectors.append(feature_vector.cpu().numpy())

# Converter os feature vectors para NumPy
feature_vectors = np.array(feature_vectors).squeeze()

# Salvar os feature vectors em um arquivo CSV
import pandas as pd
df = pd.DataFrame(feature_vectors)
df.insert(0, "image_path", image_names)  # Adiciona os caminhos das imagens
df.to_csv("feature_vectors.csv", index=False)

print("Feature vectors extraídos e salvos em feature_vectors.csv")

100%|██████████| 6.25M/6.25M [00:00<00:00, 135MB/s]


Feature vectors extraídos e salvos em feature_vectors.csv 🎉


In [15]:
df = pd.read_csv("/content/feature_vectors.csv")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df

,image_path,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,/content/the-dip/augmented_dataset/integrated_...,1.116907,1.883549e-02,3.008473,1.334610,6.635535,0.016303,4.339829,-0.055550,1.508087,1.999129,2.385495e-02,2.009096,-0.181045,1.187249,0.475453,0.588854
1,/content/the-dip/augmented_dataset/integrated_...,1.085477,2.649643e+00,1.379981,1.249477,6.705635,1.069516,4.261040,3.536861,1.391242,1.156595,2.851532e+00,2.221833,-0.221237,1.193760,-0.161058,1.399717
2,/content/the-dip/augmented_dataset/integrated_...,1.103566,-1.729075e-06,1.807996,1.388899,6.585567,0.288083,4.426752,-0.015108,1.576634,1.244862,-1.124645e-05,1.822104,-0.157716,1.124589,2.357920,1.072290
3,/content/the-dip/augmented_dataset/integrated_...,1.090405,4.445222e-02,2.422460,1.247785,6.694559,0.472502,4.258478,2.437285,1.398466,1.813676,-1.152562e-03,2.212039,-0.217444,1.200703,-0.212307,0.838325
4,/content/the-dip/augmented_dataset/integrated_...,1.102233,-2.804028e-06,2.714898,1.341638,6.635698,0.067124,4.350278,-0.142298,1.499869,1.491687,-3.242303e-05,1.980306,-0.161663,1.164588,0.505325,0.967007
5,/content/the-dip/augmented_dataset/integrated_...,1.098624,2.623150e+00,1.394623,1.260576,6.705769,1.061564,4.261610,3.520286,1.402160,1.167479,2.817447e+00,2.226589,-0.215090,1.207355,-0.159820,1.389819
6,/content/the-dip/augmented_dataset/integrated_...,1.116234,-1.406951e-01,2.337643,1.282203,6.692786,0.471021,4.266544,2.125989,1.423870,1.526973,-1.020610e-01,2.197311,-0.193878,1.221267,-0.220106,1.061982
7,/content/the-dip/augmented_dataset/integrated_...,1.116535,-1.308357e-06,2.720207,1.349904,6.630337,0.061710,4.355838,-0.118847,1.508919,1.568703,-1.702116e-05,1.975768,-0.161130,1.179179,0.644248,0.896485
8,/content/the-dip/augmented_dataset/integrated_...,1.110935,1.423670e-02,2.901211,1.331386,6.635504,0.034429,4.343714,-0.077079,1.501100,1.825304,1.842771e-02,1.999232,-0.175018,1.181016,0.503584,0.710109
9,/content/the-dip/augmented_dataset/integrated_...,1.102675,-1.416546e-06,1.823166,1.388931,6.583571,0.273181,4.429518,-0.013155,1.577450,1.249076,-6.901420e-06,1.815245,-0.156373,1.123445,2.468019,1.064091


In [25]:
# Definir os diretórios principais
image_base_dir = "/content/the-dip/augmented_dataset"  # Altere para o caminho correto das imagens
label_base_dir = "/content/the-dip/bounding_boxes_yolo/labels/train/processed_images"   # Altere para o caminho correto dos labels

# Carregar o modelo YOLOv8
model = YOLO("yolov8n.pt")  # Pode usar 'yolov8s.pt', 'yolov8m.pt', etc.

# Lista de possíveis sufixos que as imagens podem ter
image_suffixes = ["_exp", "_log", "_mean", "_std"]

# Criar um dicionário com todos os labels disponíveis
label_dict = {}

# Percorrer todas as pastas de labels e armazenar os caminhos
for root, _, files in os.walk(label_base_dir):
    for file in files:
        if file.endswith(".txt"):
            label_path = os.path.join(root, file)
            label_name = os.path.splitext(file)[0]  # Nome do arquivo sem extensão
            label_dict[label_name] = label_path  # Armazena o caminho do label

# Função para encontrar o label correspondente para uma imagem
def find_label_for_image(image_path):
    """Encontra o label correto removendo sufixos da imagem."""
    base_name = os.path.splitext(os.path.basename(image_path))[0]  # Nome sem extensão

    # Tenta encontrar o label exato primeiro
    if base_name in label_dict:
        return label_dict[base_name]

    # Se não encontrar, tenta sem os sufixos
    for suffix in image_suffixes:
        if suffix in base_name:
            base_name_no_suffix = base_name.replace(suffix, "")
            if base_name_no_suffix in label_dict:
                return label_dict[base_name_no_suffix]

    return None  # Retorna None se nenhum label correspondente for encontrado

# Função para processar as imagens e extrair features
def process_image(image_path, label_path):
    img = cv2.imread(image_path)
    if img is None:
        return None  # Ignora se a imagem não carregar

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W, _ = img.shape

    # Lê o arquivo de labels (YOLO format)
    with open(label_path, "r") as f:
        lines = f.readlines()

    feature_vectors = []
    for line in lines:
        values = line.strip().split()
        class_id = int(values[0])  # Classe do objeto (não usada aqui)

        # Convertendo coordenadas YOLO (normalizadas) para absolutas
        x_center, y_center, width, height = map(float, values[1:5])
        x1 = int((x_center - width / 2) * W)
        y1 = int((y_center - height / 2) * H)
        x2 = int((x_center + width / 2) * W)
        y2 = int((y_center + height / 2) * H)

        # Recortar a ROI (Região de Interesse)
        roi = img_rgb[y1:y2, x1:x2]
        if roi.size == 0:
            continue

        # Redimensionar a ROI para o tamanho esperado pelo modelo (640x640)
        roi_resized = cv2.resize(roi, (640, 640))

        # Converter para tensor
        roi_tensor = torch.from_numpy(roi_resized).permute(2, 0, 1).float() / 255.0
        roi_tensor = roi_tensor.unsqueeze(0)

        # Extrair feature vector do YOLOv8 (Backbone)
        backbone = model.model.model[0]
        with torch.no_grad():
            features = backbone(roi_tensor)

        # Aplicar Global Average Pooling para obter um vetor 1D
        feature_vector = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1)).reshape(features.size(0), -1)
        feature_vectors.append(feature_vector.cpu().numpy())

    return feature_vectors

# Percorrer todas as imagens e associar com labels corretos
feature_data = []

for root, _, files in os.walk(image_base_dir):  # Percorre todas as subpastas das imagens
    for file in files:
        if file.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(root, file)

            # Encontrar o label correto para a imagem
            label_path = find_label_for_image(image_path)
            if label_path is None:
                print(f"Nenhum label encontrado para {image_path}")
                continue

            # Processar a imagem e extrair features
            features = process_image(image_path, label_path)
            if features:
                for fv in features:
                    feature_data.append([image_path] + fv.tolist()[0])

# Salvar os feature vectors em um CSV
df2 = pd.DataFrame(feature_data)
df2.to_csv("/content/feature_vectors4.csv", index=False)

print("✅ Feature vectors extraídos e salvos em feature_vectors4.csv 🎉")

✅ Feature vectors extraídos e salvos em feature_vectors4.csv 🎉


In [27]:
df2 = pd.read_csv("/content/feature_vectors4.csv")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,/content/the-dip/augmented_dataset/integrated_...,1.089059,2.589928e+00,1.392454,1.250468,6.705706,1.057377,4.260878,3.516175,1.391469,1.138143,2.805044e+00,2.219826,-0.221779,1.196017,-0.162034,1.416737
1,/content/the-dip/augmented_dataset/integrated_...,1.129066,-5.692632e-06,1.842923,1.393174,6.587536,0.282009,4.422196,-0.027212,1.571948,1.246186,-3.816630e-05,1.828394,-0.160374,1.142917,2.263920,1.077753
2,/content/the-dip/augmented_dataset/integrated_...,1.094390,4.515985e-02,2.468548,1.246247,6.693007,0.449892,4.259533,2.386618,1.401040,1.826131,1.839865e-02,2.210811,-0.219700,1.197450,-0.214714,0.830606
3,/content/the-dip/augmented_dataset/integrated_...,1.114434,-2.294365e-06,2.698551,1.345926,6.632521,0.062998,4.353626,-0.127134,1.511576,1.427098,-2.045048e-05,1.967822,-0.177645,1.167506,0.579631,1.016080
4,/content/the-dip/augmented_dataset/integrated_...,1.113664,2.557747e+00,1.403690,1.260043,6.705608,1.059244,4.264048,3.487917,1.403540,1.188482,2.753812e+00,2.222957,-0.220006,1.222017,-0.159448,1.370164
5,/content/the-dip/augmented_dataset/integrated_...,1.141399,-2.413736e-01,2.521902,1.272187,6.690547,0.363917,4.270148,1.835858,1.425192,1.559484,-2.447063e-01,2.181734,-0.216038,1.237886,-0.227842,1.031010
6,/content/the-dip/augmented_dataset/integrated_...,1.152215,-1.379862e-06,2.708373,1.355641,6.628226,0.053144,4.363657,-0.100885,1.516822,1.475749,-1.095519e-05,1.954681,-0.179549,1.202144,0.769241,0.966159
7,/content/the-dip/augmented_dataset/integrated_...,1.140256,4.873865e-02,2.856192,1.337650,6.635839,0.064714,4.342232,0.091199,1.496002,1.786024,6.374580e-02,2.000037,-0.183702,1.190943,0.534342,0.746830
8,/content/the-dip/augmented_dataset/integrated_...,1.125889,-5.695400e-06,1.865144,1.391981,6.586344,0.263772,4.424997,-0.024039,1.571332,1.243660,-1.882063e-05,1.820284,-0.159287,1.139624,2.368188,1.075402
9,/content/the-dip/augmented_dataset/integrated_...,1.100099,2.595462e+00,1.391180,1.253224,6.705742,1.058305,4.261312,3.509817,1.396517,1.143147,2.805094e+00,2.219494,-0.222131,1.207551,-0.161105,1.411752
